In [4]:
# Импорт библиотек
import datetime
import requests
import json
from pybit import spot
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from pybit.exceptions import InvalidRequestError
import numpy as np
from settings import *
from Def import *
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [5]:
# Получаем данные с интервалом в 15 минут
symbol = "BTCUSD"
interval = "5"
fields = "open_time,open,high,low,close,volume"

# Время
days=12
now = datetime.datetime.now()
start_of_today = datetime.datetime(now.year, now.month, now.day)
start_of_yesterday = start_of_today - datetime.timedelta(days)

# Начальное и конечное время для запроса данных
start_time = str(int(start_of_yesterday.timestamp()))
end_time = str(int(start_of_today.timestamp()))

# Запрос первой порции данных
url = f"https://api.bybit.com/v2/public/kline/list?symbol={symbol}&interval={interval}&from={start_time}&to={end_time}&limit=200&fields={fields}"
response = requests.get(url, headers={"api-key": API, "sign": secret})
data = json.loads(response.text)
df = pd.DataFrame(data['result'])

# Запрос оставшихся данных
while len(data['result']) == 200:
    start_time = str(int(data['result'][-1]['open_time']) + 60)  # Новое начальное время
    url = f"https://api.bybit.com/v2/public/kline/list?symbol={symbol}&interval={interval}&from={start_time}&to={end_time}&limit=200&fields={fields}"
    response = requests.get(url, headers={"api-key": API, "sign": secret})
    data = json.loads(response.text)
    df = pd.concat([df, pd.DataFrame(data['result'])], axis=0, ignore_index=True)

# Преобразуем время в формат datetime
df['open_time'] = pd.to_datetime(df['open_time'], unit='s')

# Устанавливаем индекс по времени
df = df.set_index('open_time')

# Удаляем столбец "symbol"
df = df.drop('symbol', axis=1)
df = df.drop('turnover', axis=1)
df = df.drop('interval', axis=1)

# Преобразуем значения столбцов в числа с плавающей точкой и удаляем дубликаты
df = df.astype(float)
df.drop_duplicates(inplace=True)

#Создаём столбцы с ADX
df['ADX'],df['DI+'],df['DI-'] = adx(df['high'], df['low'], df['close'], n=14)
df.dropna(inplace=True)

In [6]:
# Создаем столбцы с предыдущими значениями ADX, DI+ и DI-, low
df['prev_adx'] = df['ADX'].shift()
df['prev_di_plus'] = df['DI+'].shift()
df['prev2_di_minus'] = df['DI-'].shift()
df['prev_low'] = df['low'].shift()
df['diff_adx'] = ((df['ADX'] - df['ADX'].shift(2)) + (df['ADX'] - df['ADX'].shift()))/2
df['diff_close2'] = ((df['close'] - df['close'].shift(2)) + (df['close'] - df['close'].shift(1)))/2
periods = 4
moving_average_difference(df,periods)

top_adx = 0
lowest_adx = 0
top_DI_pos= 0
#Удаляем Nan
df.dropna(inplace=True)
# Устанавливаем начальное значение флага позиции
position = 'out'

# Создаем список для хранения сигналов
signals = []
sc=[]
s=0
b=0
p=0
# Проходим по строкам таблицы
for index, row in df.iterrows():
    if row['ADX']>20 and row['ADX']>row['prev_adx']:
        top_adx=row['ADX']
    elif row['ADX']<20:
        top_adx=0

    # Условия покупки
    if position == 'out':
        if row['DI+'] - row['DI-'] > 15 and row['DI+'] > row['prev_di_plus'] and row['ADX']>20 and row['diff_adx']>2.5 and row ['volume']>4*10**6:
            b=1
            s+=1
            sc.append(s)
            p=row['close']
            print(s,b,p,row['diff_close2'],row[f'diff_close{periods}'],row['diff_adx'])
            signals.append('buy')
            position = 'long'
        else:
            signals.append('hold')
            sc.append(None)

    # Условия продажи
    elif position == 'long':
        if top_adx - row['ADX']>15:
            signals.append('sell')
            sc.append(s)
            position = 'out'
        elif row['DI-'] > row['DI+']:
            signals.append('sell')
            sc.append(s)
            position = 'out'
        else:
            signals.append('hold')
            sc.append(None)

# Создаем столбец с сигналами
df['signal'] = signals
df['score'] = sc

1 1 28496.0 297.25 0.47 3.650071613986638
2 1 28729.5 129.75 0.14 2.5756848013245346
3 1 28915.0 240.75 0.32 3.87610975225257
4 1 29295.5 104.25 0.18 5.307341984195672
5 1 29163.0 257.5 0.21 3.2795514317236716
6 1 29382.0 171.25 0.16 3.0207701064292145
7 1 29624.5 40.5 0.08 3.188010816149202
8 1 28914.0 46.5 0.09 2.8489807922403365
9 1 29120.5 110.5 0.21 3.23143418107877
10 1 29081.0 37.0 0.03 2.742098810396115
11 1 27762.5 64.25 0.08 3.3583581743778765
12 1 27943.5 238.0 0.23 3.1241239799017713
13 1 26443.5 7.0 0.04 3.1430803961193536
14 1 26710.5 40.5 0.15 3.0585299705723656
15 1 26930.0 93.75 0.09 3.651796040385298
16 1 27004.0 124.25 0.17 3.9078858559892398


In [7]:
# Устанавливаем начальную сумму денег и количество криптовалют, которые у нас есть
initial_cash = 1000
initial_cash2 = initial_cash
initial_coins = 0

# Создаем список для хранения изменений стоимости криптовалют
coin_prices = []

# Устанавливаем комиссию в 0.1%
commission = 0.0001

# Инициализируем счетчики для статистики
total_trades = 0
successful_trades = 0
unsuccessful_trades = 0

# Проходим по строкам таблицы
for index, row in df.iterrows():
    # Если есть позиция на покупку
    if position == 'long':
        # Проверяем, если есть сигнал на продажу
        if row['signal'] == 'sell':
            # Продаем криптовалюты
            coins_sold = initial_coins
            coin_prices.append(row['close'])
            initial_cash += coins_sold * row['close'] * (1 - commission)
            initial_coins = 0
            position = 'out'

            # Обновляем счетчики статистики
            total_trades += 1
            if len(coin_prices) > 1 and coin_prices[-1] > coin_prices[-2]:
                successful_trades += 1
            else:
                unsuccessful_trades += 1

    # Если нет позиции на покупку, но есть сигнал на покупку
    elif position == 'out' and row['signal'] == 'buy':
        # Покупаем криптовалюты
        coins_bought = (initial_cash * (1 - commission)) / row['close']
        coin_prices.append(row['close'])
        initial_coins += coins_bought
        initial_cash = 0
        position = 'long'

# Если остались криптовалюты в конце, продаем их по последней известной цене
if initial_coins > 0:
    coins_sold = initial_coins
    coin_prices.append(df.iloc[-1]['close'])
    initial_cash += coins_sold * df.iloc[-1]['close'] * (1 - commission)
    initial_coins = 0
    position = 'out'

# Вычисляем изменение суммы денег
profit = initial_cash - initial_cash2

# Вычисляем win rate
win_rate = (successful_trades / total_trades) * 100 if total_trades > 0 else 0

# Выводим результаты
print("initial_cash2:", initial_cash2)
print("Profit: $", profit)
print("Total Trades:", total_trades)
print("Successful Trades:", successful_trades)
print("Unsuccessful Trades:", unsuccessful_trades)
print("Win Rate: {:.2f}%".format(win_rate))
print("commission:",commission)
print("days:",days)

initial_cash2: 1000
Profit: $ -37.49568010943631
Total Trades: 16
Successful Trades: 6
Unsuccessful Trades: 10
Win Rate: 37.50%
commission: 0.0001
days: 12


In [8]:
# Создаем объект субграфика
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# Строим график свечей
candlestick = go.Candlestick(x=df.index,
                             open=df['open'],
                             high=df['high'],
                             low=df['low'],
                             close=df['close'],
                             name='candlestick')
fig.add_trace(candlestick, row=1, col=1)

# Строим график объема
bar = go.Bar(x=df.index,
             y=df['volume'],
             name='volume',
             marker=dict(color=df['close'] - df['open'],
                         colorscale='RdYlGn',
                         line=dict(color='gray', width=0.5)),
             opacity=0.8)
fig.add_trace(bar, row=2, col=1)

# Строим график ADX
adx = go.Scatter(x=df.index,
                 y=df['ADX'],
                 name='ADX',
                 line=dict(color='blue', width=1),
                 opacity=0.8)
fig.add_trace(adx, row=3, col=1)

# Строим график DI+
dip = go.Scatter(x=df.index,
                 y=df['DI+'],
                 name='DI+',
                 line=dict(color='green', width=1),
                 opacity=0.8)
fig.add_trace(dip, row=3, col=1)

# Строим график DI-
din = go.Scatter(x=df.index,
                 y=df['DI-'],
                 name='DI-',
                 line=dict(color='red', width=1),
                 opacity=0.8)
fig.add_trace(din, row=3, col=1)

# Создаем списки с координатами сигналов покупки и продажи
buy_signals = df.loc[df['signal'] == 'buy']
sell_signals = df.loc[df['signal'] == 'sell']

# Строим график сигналов покупки и продажи
if not buy_signals.empty:
    buy_scatter = go.Scatter(x=buy_signals.index, y=buy_signals['close'],
                         name='buy', mode='markers', 
                         marker=dict(color='green', size=10),
                         hovertext=buy_signals['score'])
    fig.add_trace(buy_scatter, row=1, col=1)
if not sell_signals.empty:
    sell_scatter = go.Scatter(x=sell_signals.index, y=sell_signals['close'],
                          name='sell', mode='markers', 
                          marker=dict(color='red', size=10),
                          hovertext=sell_signals['score'])
    fig.add_trace(sell_scatter, row=1, col=1)

# Настраиваем макет графика
fig.update_layout(title='BTCUSD',
                  yaxis_title='Price, USD',
                  height=1000,
                  xaxis_rangeslider_visible=True)

# Отображаем график
fig.show()